In [ ]:
import numpy as np
import cv2
from matplotlib import pyplot as plt
from numba import jit
from tqdm import tqdm_notebook

from PIL import Image
from __future__ import print_function
from skimage.feature import peak_local_max
from skimage.morphology import watershed
from scipy import ndimage
import argparse
import imutils

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!ls /content/gdrive/MyDrive/NorNickel/dataset1-1/

F1_1_1_1.ts  F1_1_3_2.ts  F1_2_2_1.ts  F1_2_4_2.ts  F2_1_2_1.ts  F2_2_2_2.ts
F1_1_1_2.ts  F1_1_4_1.ts  F1_2_2_2.ts  F1_2_5_1.ts  F2_1_2_2.ts  F2_2_3_1.ts
F1_1_2_1.ts  F1_1_4_2.ts  F1_2_3_1.ts  F1_2_5_2.ts  F2_2_1_1.ts  F2_2_3_2.ts
F1_1_2_2.ts  F1_1_5_1.ts  F1_2_3_2.ts  F2_1_1_1.ts  F2_2_1_2.ts  __MACOSX
F1_1_3_1.ts  F1_1_5_2.ts  F1_2_4_1.ts  F2_1_1_2.ts  F2_2_2_1.ts


In [ ]:
import os
def mkdir_ifnotexists(dir):
    if os.path.exists(dir):
        return
    os.mkdir(dir)

In [ ]:
%%time
vid_file='/content/gdrive/MyDrive/NorNickel/dataset1-1/F1_1_1_1.ts'
frame_pth='./frames'
mkdir_ifnotexists(frame_pth)
cmd = "ffmpeg -i %s -start_number 0 -vsync 0 %s/frame_%%06d.png" % (
            vid_file,
            frame_pth,
        )
os.system(cmd)

CPU times: user 208 ms, sys: 25.5 ms, total: 234 ms
Wall time: 1min 8s


In [ ]:
# @jit(nopython=True)
# def sobelOperator(img):
#     container = np.copy(img)
#     size = container.shape
#     for i in range(1, size[0] - 1):
#         for j in range(1, size[1] - 1):
#             gx = (img[i - 1][j - 1] + 2*img[i][j - 1] + img[i + 1][j - 1]) - (img[i - 1][j + 1] + 2*img[i][j + 1] + img[i + 1][j + 1])
#             gy = (img[i - 1][j - 1] + 2*img[i - 1][j] + img[i - 1][j + 1]) - (img[i + 1][j - 1] + 2*img[i + 1][j] + img[i + 1][j + 1])
#             container[i][j] = min(255, np.sqrt(gx**2 + gy**2))
#     return container

In [ ]:
# mkdir_ifnotexists('./filtered_imgs')

# for image_name in tqdm_notebook(os.listdir('./frames')):
#     img = cv2.cvtColor(cv2.imread(f"./frames/{image_name}"), cv2.COLOR_BGR2GRAY)
#     img = sobelOperator(img)
#     img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
#     cv2.imwrite(f'./filtered_imgs/{image_name}', img)

In [ ]:
def visualize(imgs, format=None, gray=False):
    plt.figure(figsize=(20, 40))
    for i, img in enumerate(imgs):
        if img.shape[0] == 3:
            img = img.transpose(1,2,0)
        plt_idx = i+1
        plt.subplot(2, 2, plt_idx)
        plt.imshow(img, format)
    plt.show()

In [ ]:
from scipy import ndimage
from scipy.ndimage.filters import convolve

from scipy import misc
import numpy as np

class cannyEdgeDetector:
    def __init__(self, imgs, sigma=1, kernel_size=5, weak_pixel=75, strong_pixel=255, lowthreshold=0.05, highthreshold=0.15):
        self.imgs = imgs
        self.imgs_final = []
        self.img_smoothed = None
        self.gradientMat = None
        self.thetaMat = None
        self.nonMaxImg = None
        self.thresholdImg = None
        self.weak_pixel = weak_pixel
        self.strong_pixel = strong_pixel
        self.sigma = sigma
        self.kernel_size = kernel_size
        self.lowThreshold = lowthreshold
        self.highThreshold = highthreshold
        return 
    
    def gaussian_kernel(self, size, sigma=1):
        size = int(size) // 2
        x, y = np.mgrid[-size:size+1, -size:size+1]
        normal = 1 / (2.0 * np.pi * sigma**2)
        g =  np.exp(-((x**2 + y**2) / (2.0*sigma**2))) * normal
        return g
    
    def sobel_filters(self, img):
        Kx = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]], np.float32)
        Ky = np.array([[1, 2, 1], [0, 0, 0], [-1, -2, -1]], np.float32)

        Ix = ndimage.filters.convolve(img, Kx)
        Iy = ndimage.filters.convolve(img, Ky)

        G = np.hypot(Ix, Iy)
        G = G / G.max() * 255
        theta = np.arctan2(Iy, Ix)
        return (G, theta)
    
    def non_max_suppression(self, img, D):
        M, N = img.shape
        Z = np.zeros((M,N), dtype=np.int32)
        angle = D * 180. / np.pi
        angle[angle < 0] += 180


        for i in range(1,M-1):
            for j in range(1,N-1):
                try:
                    q = 255
                    r = 255

                   #angle 0
                    if (0 <= angle[i,j] < 22.5) or (157.5 <= angle[i,j] <= 180):
                        q = img[i, j+1]
                        r = img[i, j-1]
                    #angle 45
                    elif (22.5 <= angle[i,j] < 67.5):
                        q = img[i+1, j-1]
                        r = img[i-1, j+1]
                    #angle 90
                    elif (67.5 <= angle[i,j] < 112.5):
                        q = img[i+1, j]
                        r = img[i-1, j]
                    #angle 135
                    elif (112.5 <= angle[i,j] < 157.5):
                        q = img[i-1, j-1]
                        r = img[i+1, j+1]

                    if (img[i,j] >= q) and (img[i,j] >= r):
                        Z[i,j] = img[i,j]
                    else:
                        Z[i,j] = 0


                except IndexError as e:
                    pass

        return Z

    def threshold(self, img):

        highThreshold = img.max() * self.highThreshold;
        lowThreshold = highThreshold * self.lowThreshold;

        M, N = img.shape
        res = np.zeros((M,N), dtype=np.int32)

        weak = np.int32(self.weak_pixel)
        strong = np.int32(self.strong_pixel)

        strong_i, strong_j = np.where(img >= highThreshold)
        zeros_i, zeros_j = np.where(img < lowThreshold)

        weak_i, weak_j = np.where((img <= highThreshold) & (img >= lowThreshold))

        res[strong_i, strong_j] = strong
        res[weak_i, weak_j] = weak

        return (res)

    def hysteresis(self, img):

        M, N = img.shape
        weak = self.weak_pixel
        strong = self.strong_pixel

        for i in range(1, M-1):
            for j in range(1, N-1):
                if (img[i,j] == weak):
                    try:
                        if ((img[i+1, j-1] == strong) or (img[i+1, j] == strong) or (img[i+1, j+1] == strong)
                            or (img[i, j-1] == strong) or (img[i, j+1] == strong)
                            or (img[i-1, j-1] == strong) or (img[i-1, j] == strong) or (img[i-1, j+1] == strong)):
                            img[i, j] = strong
                        else:
                            img[i, j] = 0
                    except IndexError as e:
                        pass

        return img
    
    def detect(self):
        imgs_final = []
        for img_num in tqdm_notebook(range(len(self.imgs))):
            if img_num % 10 == 0:   
                self.img_smoothed = convolve(self.imgs[img_num], self.gaussian_kernel(self.kernel_size, self.sigma))
                self.gradientMat, self.thetaMat = self.sobel_filters(self.img_smoothed)
                self.nonMaxImg = self.non_max_suppression(self.gradientMat, self.thetaMat)
                self.thresholdImg = self.threshold(self.nonMaxImg)
                img_final = self.hysteresis(self.thresholdImg)
                self.imgs_final.append(img_final)

        return self.imgs_final

In [ ]:
%%time
imgs = [cv2.cvtColor(cv2.imread(f"./frames/{image_name}"), cv2.COLOR_BGR2GRAY) for image_name in os.listdir('./frames')]

CPU times: user 18.2 s, sys: 851 ms, total: 19.1 s
Wall time: 17.2 s


In [ ]:
detector = cannyEdgeDetector(imgs, sigma=10, kernel_size=5, lowthreshold=0.7, highthreshold=0.8, weak_pixel=180)

In [ ]:
imgs_final = detector.detect()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:128: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


In [ ]:
mkdir_ifnotexists('./filtered')
for id in range(len(imgs_final)):
    image = Image.fromarray(imgs_final[id])
    cv2.imwrite(f'./filtered/frame_{id}.png', np.float32(image))

In [ ]:
os.system('ffmpeg -r 10 -i ./filtered/frame_%d.png -vcodec libx264 -b 10M -y /content/new.mp4')

In [ ]:
!zip -r ./frames.zip ./frames/

  adding: frames/ (stored 0%)
  adding: frames/frame_000550.png (deflated 0%)
  adding: frames/frame_001270.png (deflated 0%)
  adding: frames/frame_000864.png (deflated 0%)
  adding: frames/frame_000829.png (deflated 0%)
  adding: frames/frame_000390.png (deflated 0%)
  adding: frames/frame_000903.png (deflated 0%)
  adding: frames/frame_000152.png (deflated 0%)
  adding: frames/frame_001625.png (deflated 0%)
  adding: frames/frame_001268.png (deflated 0%)
  adding: frames/frame_001326.png (deflated 0%)
  adding: frames/frame_001203.png (deflated 0%)
  adding: frames/frame_001305.png (deflated 0%)
  adding: frames/frame_000072.png (deflated 0%)
  adding: frames/frame_000138.png (deflated 0%)
  adding: frames/frame_000273.png (deflated 0%)
  adding: frames/frame_000961.png (deflated 0%)
  adding: frames/frame_000022.png (deflated 0%)
  adding: frames/frame_001661.png (deflated 0%)
  adding: frames/frame_001245.png (deflated 0%)
  adding: frames/frame_001737.png (deflated 0%)
  adding: 